# Homework

Clone the course repo if you haven't:

git clone https://github.com/alexeygrigorev/mlbookcamp-code.git
Go to the course-zoomcamp/cohorts/2022/05-deployment/homework folder and execute the following:

```bash
docker build -t zoomcamp-model:v001 .

Note: If you have troubles building the image, you can use the image we built and published to docker hub: svizor42/zoomcamp-model:v001

### Question 1
Run it to test that it's working locally:

```bash
docker run -it --rm -p 9696:9696 zoomcamp-model:v001
```
And in another terminal, execute q6_test.py file:

```bash 
python q6_test.py

In [1]:
!ls

Dockerfile       homework10.ipynb q6_test.py


In [3]:
!docker build -t zoomcamp-model:v001 .

[+] Building 0.0s (0/1)                                                         
[+] Building 0.2s (2/3)                                                         
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 37B                                        0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for docker.io/svizor/zoomcamp-model:3.9.12-s  0.1s
[+] Building 0.3s (2/3)                                                         
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 37B                                        0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for dock

In [4]:
!python q6_test.py

{'get_card': True, 'get_card_probability': 0.7692649226628628}


The answer is 0.769.

### Question 2
What's the version of kind that you have?

Use `kind --version` to find out.

In [5]:
!kind --version

kind version 0.17.0


In [31]:
!kind create cluster

Creating cluster "kind" ...
 ✓ Ensuring node image (kindest/node:v1.25.3) 🖼
 ✓ Preparing nodes 📦 7l
 ✓ Writing configuration 📜7l
 ✓ Starting control-plane 🕹️7l
 ✓ Installing CNI 🔌7l
 ✓ Installing StorageClass 💾7l
Set kubectl context to "kind-kind"
You can now use your cluster with:

kubectl cluster-info --context kind-kind

Have a question, bug, or feature request? Let us know! https://kind.sigs.k8s.io/#community 🙂


In [32]:
!kubectl cluster-info

Kubernetes control plane is running at https://127.0.0.1:60521
CoreDNS is running at https://127.0.0.1:60521/api/v1/namespaces/kube-system/services/kube-dns:dns/proxy

To further debug and diagnose cluster problems, use 'kubectl cluster-info dump'.


In [9]:
!kubectl cluster-info --context kind-hm10-cluster

Kubernetes control plane is running at https://127.0.0.1:55973
CoreDNS is running at https://127.0.0.1:55973/api/v1/namespaces/kube-system/services/kube-dns:dns/proxy

To further debug and diagnose cluster problems, use 'kubectl cluster-info dump'.


### Question 3
What's the smallest deployable computing unit that we can create and manage in Kubernetes (kind in our case)?

The answer is `pod`.

In [33]:
!kubectl get svc

NAME         TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   28s


### Question 4
Now let's test if everything works. Use kubectl to get the list of running services.

What's the Type of the service that is already running there?

Answer: `ClusterIP`

### Question 5
To be able to use the docker image we previously created (zoomcamp-model:v001), we need to register it with kind.

What's the command we need to run for that?

In [35]:
!kind load docker-image zoomcamp-model:v001

Image: "zoomcamp-model:v001" with ID "sha256:e96ca63188e465d80de65f80c73ffbabd612f84236faa5cb1a6b836551dc9034" found to be already present on all nodes.


Answer: `kind load docker-image`

### Question 6
Now let's create a deployment config (e.g. deployment.yaml):

```bash
apiVersion: apps/v1
kind: Deployment
metadata:
  name: credit-card
spec:
  selector:
    matchLabels:
      app: credit-card
  replicas: 1
  template:
    metadata:
      labels:
        app: credit-card
    spec:
      containers:
      - name: credit-card
        image: <Image>
        resources:
          requests:
            memory: "64Mi"
            cpu: "100m"            
          limits:
            memory: <Memory>
            cpu: <CPU>
        ports:
        - containerPort: <Port>
```
Replace `<Image>, <Memory>, <CPU>, <Port>` with the correct values.

What is the value for `<Port>`?

Apply this deployment using the appropriate command and get a list of running Pods. You can see one running Pod.

In [36]:
!kubectl apply -f deployment.yaml

deployment.apps/credit-card created


In [41]:
!kubectl get pod

NAME                          READY   STATUS    RESTARTS   AGE
credit-card-59fb69bdf-j64qz   1/1     Running   0          59s


In [42]:
!kubectl describe pod credit-card-59fb69bdf-j64qz

Name:             credit-card-59fb69bdf-j64qz
Namespace:        default
Priority:         0
Service Account:  default
Node:             kind-control-plane/172.20.0.2
Start Time:       Wed, 07 Dec 2022 19:47:15 +0200
Labels:           app=credit-card
                  pod-template-hash=59fb69bdf
Annotations:      <none>
Status:           Running
IP:               10.244.0.5
IPs:
  IP:           10.244.0.5
Controlled By:  ReplicaSet/credit-card-59fb69bdf
Containers:
  credit-card:
    Container ID:   containerd://466be32852552df04808c0b94c486ee59dc097c70d33498b3e3bc70158d01625
    Image:          zoomcamp-model:v001
    Image ID:       docker.io/library/import-2022-12-07@sha256:be54f21b3b034aff7ae7161b2b8bbdb1eeb7dcd91da69b06cc2d87c842ce1549
    Port:           9696/TCP
    Host Port:      0/TCP
    State:          Running
      Started:      Wed, 07 Dec 2022 19:47:17 +0200
    Ready:          True
    Restart Count:  0
    Limits:
      cpu:     100m
      memory:  64Mi
    Requests:
  

### Question 7
Let's create a service for this deployment (service.yaml):

```bash
apiVersion: v1
kind: Service
metadata:
  name: <Service name>
spec:
  type: LoadBalancer
  selector:
    app: <???>
  ports:
  - port: 80
    targetPort: <PORT>


Fill it in. What do we need to write instead of <???>?

Apply this config file.


Answer: `credit-card`



In [43]:
!kubectl apply -f service.yaml

service/credit-card created


In [ ]:
!kubectl port-forward service/credit-card 9696:80 in terminal

In [45]:
!python q6_test.py #testing the that kubernetes is running and can be reached with the service

{'get_card': True, 'get_card_probability': 0.7692649226628628}


### Autoscaling
Now we're going to use a HorizontalPodAutoscaler (HPA for short) that automatically updates a workload resource (such as our deployment), with the aim of automatically scaling the workload to match demand.

Use the following command to create the HPA:

```bash
kubectl autoscale deployment credit-card --name credit-card-hpa --cpu-percent=20 --min=1 --max=3

In [46]:
!kubectl autoscale deployment credit-card --name credit-card-hpa --cpu-percent=20 --min=1 --max=3

horizontalpodautoscaler.autoscaling/credit-card-hpa autoscaled


You can check the current status of the new HPA by running:

```bash
kubectl get hpa

In [51]:
!kubectl get hpa

NAME              REFERENCE                TARGETS         MINPODS   MAXPODS   REPLICAS   AGE
credit-card-hpa   Deployment/credit-card   <unknown>/20%   1         3         1          3m49s


In [50]:
!kubectl apply -f https://github.com/kubernetes-sigs/metrics-server/releases/latest/download/components.yaml

serviceaccount/metrics-server created
clusterrole.rbac.authorization.k8s.io/system:aggregated-metrics-reader created
clusterrole.rbac.authorization.k8s.io/system:metrics-server created
rolebinding.rbac.authorization.k8s.io/metrics-server-auth-reader created
clusterrolebinding.rbac.authorization.k8s.io/metrics-server:system:auth-delegator created
clusterrolebinding.rbac.authorization.k8s.io/system:metrics-server created
service/metrics-server created
deployment.apps/metrics-server created
apiservice.apiregistration.k8s.io/v1beta1.metrics.k8s.io created


In [53]:
!!python q6_test.py

^C


[]